In [1]:
import tensorflow as tf
from tensorflow.python.keras.models import sequential
from tensorflow.python.keras import layers
import os
from tensorflow.python import keras 
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import pickle
import random as rnd
import numpy as np
tf.keras.applications
from tensorflow.keras.applications import ResNet50

In [8]:
data_dir="raw_img"
img_size=192
train_dataset = []

for classes in os.listdir(data_dir):
    for images in tqdm(os.listdir(os.path.join(data_dir,classes))):
        image = cv2.imread(data_dir+"/"+classes+"/"+images)
        train_dataset.append([cv2.resize(image,(img_size,img_size)),os.listdir(data_dir).index(classes)])

100%|█████████████████████████████████████████████████████████████████████████████| 1862/1862 [00:17<00:00, 105.72it/s]


In [ ]:
os.makedirs("Variables", exist_ok=True)
with open("Variables/train_dataset.pickle", "wb") as f:
    pickle.dump(train_dataset, f)

In [ ]:
train_dataset = pickle.load(open("variables/train_dataset.pickle","rb"))

In [ ]:
rnd.shuffle(train_dataset)
X = []
Y = []


In [ ]:
for features, labels in train_dataset:
    X.append(features)
    Y.append(labels)

In [ ]:
print(Y[:10])

In [ ]:
x_train=np.array(X)
y_train=np.array(Y)
print(x_train.shape)
print(y_train.shape)


In [20]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [27]:
# Create a new model by adding custom layers on top of ResNet50 base
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

for layer in base_model.layers:
    layer.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.Dense(512, activation='relu'))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [30]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 4, 4, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 4, 4, 512)           │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,641,930 (94.00 MB)

 Trainable params: 1,054,218 (4.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [32]:
model.fit(x_train, y_train, epochs=10, batch_size = 32, validation_split = 0.3)

Epoch 1/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 1369s 2s/step - accuracy: 0.9549 - loss: 0.1394 - val_accuracy: 0.9088 - val_loss: 0.3186
Epoch 2/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 1000s 2s/step - accuracy: 0.9607 - loss: 0.1184 - val_accuracy: 0.9128 - val_loss: 0.3589
Epoch 3/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 998s 2s/step - accuracy: 0.9604 - loss: 0.1183 - val_accuracy: 0.9137 - val_loss: 0.3316
Epoch 4/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 1015s 2s/step - accuracy: 0.9689 - loss: 0.0957 - val_accuracy: 0.9123 - val_loss: 0.3638
Epoch 5/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 1006s 2s/step - accuracy: 0.9677 - loss: 0.1019 - val_accuracy: 0.9071 - val_loss: 0.4122
Epoch 6/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 997s 2s/step - accuracy: 0.9723 - loss: 0.0867 - val_accuracy: 0.9113 - val_loss: 0.3813
Epoch 7/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 999s 2s/step - accuracy: 0.9729 - loss: 0.0803 - val_accuracy: 0.9102 - val_loss: 0.3882
Epoch 8/10
573/573 ━━━━━━━━━━━━━━━━━━━━ 997s 2s/step - accuracy: 0.9771 - loss: 0.0690 - val_

In [47]:
model.save("Animal_Classification_Model.keras")
